**<font size = 5 > 0. SETUP LIBRARIES AND PACKAGES** </font>

In [1]:
#####--- 0.1 Import libraries
from kafka import KafkaConsumer
import json
from pymongo import MongoClient
from pathlib import Path
from ruamel.yaml import YAML
import time
from  datetime import datetime
import warnings
warnings.filterwarnings('ignore')

**<font size = 5 > 1. IMPORT CONFIG_FILE AND SET DB** </font>

In [ ]:
#####--- 1.0 Read Config_File and set some variables
yaml        = YAML(typ='safe')
with open('Config_File.yml', 'r') as fil:
        config = yaml.load(fil)
config_general = config["general"]
config_pgr = config["pgr1"]
act_topic = config_pgr["topic"]
hours_gtm = int(config_general["hours_gtm"])

#####--- 1.1 Connect to DB
client = MongoClient('mongo', 27017, username='', password='')
db = client.dataman 

#####--- 1.2 Set consumer
consumer = KafkaConsumer(
  bootstrap_servers=["kafka:9092"],
  auto_offset_reset='earliest',
  value_deserializer=lambda m: json.loads(m.decode('utf-8'))
  )



**<font size = 5 > 2. CONSUME DATA** </font>

In [ ]:
#####--- 2.0 Subscribe topic
consumer.subscribe(act_topic) 

#####--- 2.1 Consume data
while(True):
    for message in consumer:
        try:
            messaggio = message.value
            ora = messaggio['created_at']
            ora = datetime.strptime(ora, "%Y-%m-%d %H:%M:%S")
            ora = datetime(ora.year, ora.month, ora.day, 
                   ora.hour + hours_gtm, ora.minute, ora.second) 
            messaggio['created_at'] = ora
            db.qr.insert_one(messaggio) 
        except:
            print("errore")
    time.sleep(5)